In [5]:
from joblib import Parallel, delayed
import os
import numpy as np

In [6]:
from Dissertation.Const import Dissertation_Const_1D

In [7]:
failedruns5 = []

def save_results(model, base_dir="dissertation_const", save_stride=20):
    os.makedirs(base_dir, exist_ok=True)
    fname = os.path.join(base_dir, f"const_{model.const:.2f}_lambda_{model.k:.2f}_m0_{model.m0:.2f}_n0_{model.n0:.2f}.npz")

    indices = np.arange(0, len(model.times), save_stride)
    np.savez_compressed(fname,
                        x=model.x,
                        t=model.times[indices],
                        N_arr=model.N_arr[indices],
                        M_arr=model.M_arr[indices],
                        wave_speed=model.wave_speed)

def run_and_save_single(const, lam, m0, n0, base_dir="dissertation_const"):
    print(f"▶ Running: α = {const}, λ = {lam}, m₀ = {m0}, n₀ = {n0}")
    model = Dissertation_Const_1D(
        D=1.0, rho=1.0, K=1.0,
        k=lam, const=const,
        n0=n0, m0=m0,
        L=1000.0, N=5001,
        T=550, dt=0.1,
        scheme="AB2AM2",
        init_type="tanh",
        steepness=0.1
    )
    model.solve()
    speed, _, _ = model.estimate_wave_speed(plot=False)

    if speed is None:
        print(f"❌ Failed: α = {const}, λ = {lam}, m₀ = {m0}, n₀ = {n0}")
        return (const, lam, m0, n0, False)
    else:
        model.wave_speed = speed
        save_results(model, base_dir=base_dir)
        print(f"✅ Saved: α = {const}, λ = {lam}, m₀ = {m0}, n₀ = {n0}")
        return (const, lam, m0, n0, True)

def run_parallel_batch():
    const_vals  = [0.1, 0.3, 0.5, 1.0, 2, 5, 10, 50, 100]
    lambda_vals = [0.1, 0.2, 0.5, 0.8, 1.0, 2, 5, 10, 50, 100]
    m0_vals     = [0, 0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 0.95, 1.0]
    n0_vals     = [0.5, 1.0]

    param_grid = [(const, lam, m0, n0) for const in const_vals for lam in lambda_vals for m0 in m0_vals for n0 in n0_vals]

    results = Parallel(n_jobs=-2, verbose=10)(
        delayed(run_and_save_single)(const, lam, m0, n0) for const, lam, m0, n0 in param_grid
    )

    global failedruns5
    failedruns5 = [(const, lam, m0, n0) for (const, lam, m0, n0, success) in results if not success]
    if failedruns5:
        print("\n❌ The following runs failed to compute wave speed:")
        with open("failed_runs_dissertation_const.txt", "w") as f:
            for const, lam, m0, n0 in failedruns5:
                print(f"  α = {const}, λ = {lam}, m₀ = {m0}, n₀ = {n0}")
                f.write(f"const = {const}, lambda = {lam}, m0 = {m0}, n0 = {n0}\n")
    else:
        print("\n✅ All runs succeeded.")

if __name__ == "__main__":
    run_parallel_batch()

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.


▶ Running: α = 0.1, λ = 0.1, m₀ = 0.1, n₀ = 1.0
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.05, n₀ = 0.5
▶ Running: α = 0.1, λ = 0.1, m₀ = 0, n₀ = 0.5
▶ Running: α = 0.1, λ = 0.1, m₀ = 0, n₀ = 1.0
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.05, n₀ = 1.0
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.1, n₀ = 0.5
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.2, n₀ = 0.5
Estimated speed = 0.2456, R² = 0.9913
Estimated speed = 0.2483, R² = 0.9901
Estimated speed = 0.2415, R² = 0.9898
Estimated speed = 0.2490, R² = 0.9911
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.05, n₀ = 1.0
Estimated speed = 0.2547, R² = 0.9913
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.2, n₀ = 0.5
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0, n₀ = 0.5
Estimated speed = 0.2433, R² = 0.9892
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.2, n₀ = 1.0
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.5, n₀ = 0.5
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.1, n₀ = 1.0
Estimated speed = 0.2524, R² = 0.9911
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.5, n₀ = 1.0


[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:  1.1min


▶ Running: α = 0.1, λ = 0.1, m₀ = 0.8, n₀ = 0.5
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.05, n₀ = 0.5
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.1, n₀ = 0.5
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.8, n₀ = 1.0
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0, n₀ = 1.0
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.9, n₀ = 0.5
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.9, n₀ = 1.0
Estimated speed = 0.2182, R² = 0.9953
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.5, n₀ = 0.5
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.95, n₀ = 0.5
Estimated speed = 0.2357, R² = 0.9902
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.2, n₀ = 1.0
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.95, n₀ = 1.0
Estimated speed = 0.2285, R² = 0.9967
❌ Not enough valid front points.
❌ Failed: α = 0.1, λ = 0.1, m₀ = 0.8, n₀ = 0.5
▶ Running: α = 0.1, λ = 0.1, m₀ = 1.0, n₀ = 0.5
❌ Not enough valid front points.
❌ Failed: α = 0.1, λ = 0.1, m₀ = 0.8, n₀ = 1.0
❌ Not enough valid front points.
❌ Failed: α = 0.1, λ = 0.1, m₀ = 0.9, n₀ = 0.5
❌ Not enough valid front points.
▶ Running: α = 0.1, λ = 0.1, m₀ = 1.0, n₀ = 1.

[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:  2.0min


...

[Parallel(n_jobs=-2)]: Done 1499 tasks      | elapsed: 168.3min


❌ Not enough valid front points.
❌ Failed: α = 100, λ = 0.8, m₀ = 0.1, n₀ = 1.0
▶ Running: α = 100, λ = 0.8, m₀ = 0.9, n₀ = 0.5
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 0.8, m₀ = 0.5, n₀ = 0.5
▶ Running: α = 100, λ = 0.8, m₀ = 0.9, n₀ = 1.0
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 0.8, m₀ = 0.2, n₀ = 0.5
▶ Running: α = 100, λ = 0.8, m₀ = 0.95, n₀ = 0.5
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 0.8, m₀ = 0.2, n₀ = 1.0
▶ Running: α = 100, λ = 0.8, m₀ = 0.95, n₀ = 1.0
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 0.8, m₀ = 0.5, n₀ = 1.0
▶ Running: α = 100, λ = 0.8, m₀ = 1.0, n₀ = 0.5
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 0.8, m₀ = 0.8, n₀ = 0.5
▶ Running: α = 100, λ = 0.8, m₀ = 1.0, n₀ = 1.0
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 0.8, m₀ = 0.8, n₀ = 1.0
▶ Running: α = 100, λ = 1.0, m₀ = 0, n₀ = 0.5
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 0.8, m₀ = 0.9, n₀ = 0.5
▶ Running: α = 100, λ = 

[Parallel(n_jobs=-2)]: Done 1554 tasks      | elapsed: 173.6min


❌ Not enough valid front points.
❌ Failed: α = 100, λ = 5, m₀ = 0.2, n₀ = 0.5
▶ Running: α = 100, λ = 5, m₀ = 0.9, n₀ = 1.0
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 5, m₀ = 0.2, n₀ = 1.0
▶ Running: α = 100, λ = 5, m₀ = 0.95, n₀ = 0.5
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 5, m₀ = 0.5, n₀ = 0.5
▶ Running: α = 100, λ = 5, m₀ = 0.95, n₀ = 1.0
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 5, m₀ = 0.5, n₀ = 1.0
▶ Running: α = 100, λ = 5, m₀ = 1.0, n₀ = 0.5
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 5, m₀ = 0.8, n₀ = 1.0
▶ Running: α = 100, λ = 5, m₀ = 1.0, n₀ = 1.0
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 5, m₀ = 0.8, n₀ = 0.5
▶ Running: α = 100, λ = 10, m₀ = 0, n₀ = 0.5
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 5, m₀ = 0.9, n₀ = 0.5
▶ Running: α = 100, λ = 10, m₀ = 0, n₀ = 1.0
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 5, m₀ = 0.9, n₀ = 1.0
▶ Running: α = 100, λ = 10, m₀ = 0.05, n₀ = 0.5
❌ Not 

[Parallel(n_jobs=-2)]: Done 1620 out of 1620 | elapsed: 180.4min finished


In [8]:
from Chloe.Const import Chloe_Const_1D

In [9]:
failedruns6 = []

def save_results(model, base_dir="chloe_const", save_stride=20):
    os.makedirs(base_dir, exist_ok=True)
    fname = os.path.join(base_dir, f"const_{model.const:.2f}_lambda_{model.k:.2f}_m0_{model.m0:.2f}_n0_{model.n0:.2f}.npz")

    indices = np.arange(0, len(model.times), save_stride)
    np.savez_compressed(fname,
                        x=model.x,
                        t=model.times[indices],
                        N_arr=model.N_arr[indices],
                        M_arr=model.M_arr[indices],
                        wave_speed=model.wave_speed)

def run_and_save_single(const, lam, m0, n0, base_dir="chloe_const"):
    print(f"▶ Running: α = {const}, λ = {lam}, m₀ = {m0}, n₀ = {n0}")
    model = Dissertation_Const_1D(
        D=1.0, rho=1.0, K=1.0,
        k=lam, const=const,
        n0=n0, m0=m0,
        L=1000.0, N=5001,
        T=550, dt=0.1,
        scheme="AB2AM2",
        init_type="tanh",
        steepness=0.1
    )
    model.solve()
    speed, _, _ = model.estimate_wave_speed(plot=False)

    if speed is None:
        print(f"❌ Failed: α = {const}, λ = {lam}, m₀ = {m0}, n₀ = {n0}")
        return (const, lam, m0, n0, False)
    else:
        model.wave_speed = speed
        save_results(model, base_dir=base_dir)
        print(f"✅ Saved: α = {const}, λ = {lam}, m₀ = {m0}, n₀ = {n0}")
        return (const, lam, m0, n0, True)

def run_parallel_batch():
    const_vals  = [0.1, 0.3, 0.5, 1.0, 2, 5, 10, 50, 100]
    lambda_vals = [0.1, 0.2, 0.5, 0.8, 1.0, 2, 5, 10, 50, 100]
    m0_vals     = [0, 0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 0.95, 1.0]
    n0_vals     = [0.5, 1.0]

    param_grid = [(const, lam, m0, n0) for const in const_vals for lam in lambda_vals for m0 in m0_vals for n0 in n0_vals]

    results = Parallel(n_jobs=-1, verbose=10)(
        delayed(run_and_save_single)(const, lam, m0, n0) for const, lam, m0, n0 in param_grid
    )

    global failedruns6
    failedruns6 = [(const, lam, m0, n0) for (const, lam, m0, n0, success) in results if not success]
    if failedruns6:
        print("\n❌ The following runs failed to compute wave speed:")
        with open("failed_runs_chloe_const.txt", "w") as f:
            for const, lam, m0, n0 in failedruns6:
                print(f"  α = {const}, λ = {lam}, m₀ = {m0}, n₀ = {n0}")
                f.write(f"const = {const}, lambda = {lam}, m0 = {m0}, n0 = {n0}\n")
    else:
        print("\n✅ All runs succeeded.")

if __name__ == "__main__":
    run_parallel_batch()

▶ Running: α = 0.1, λ = 0.1, m₀ = 0, n₀ = 0.5
▶ Running: α = 0.1, λ = 0.1, m₀ = 0, n₀ = 1.0
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.05, n₀ = 0.5
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.05, n₀ = 1.0
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.1, n₀ = 1.0
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.1, n₀ = 0.5
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.2, n₀ = 0.5


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


▶ Running: α = 0.1, λ = 0.1, m₀ = 0.2, n₀ = 1.0
Estimated speed = 0.2483, R² = 0.9901
Estimated speed = 0.2456, R² = 0.9913
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.2, n₀ = 0.5
Estimated speed = 0.2490, R² = 0.9911
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.5, n₀ = 0.5
Estimated speed = 0.2415, R² = 0.9898
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.05, n₀ = 1.0
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.5, n₀ = 1.0


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   47.6s


Estimated speed = 0.2433, R² = 0.9892
Estimated speed = 0.2547, R² = 0.9913
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.1, n₀ = 1.0
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0, n₀ = 0.5
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.8, n₀ = 0.5
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.8, n₀ = 1.0
Estimated speed = 0.2524, R² = 0.9911
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.05, n₀ = 0.5
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.1, n₀ = 0.5
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.9, n₀ = 0.5
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.9, n₀ = 1.0
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0, n₀ = 1.0
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.95, n₀ = 0.5
Estimated speed = 0.2357, R² = 0.9902
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.2, n₀ = 1.0
▶ Running: α = 0.1, λ = 0.1, m₀ = 0.95, n₀ = 1.0
Estimated speed = 0.2285, R² = 0.9967
Estimated speed = 0.2182, R² = 0.9953
✅ Saved: α = 0.1, λ = 0.1, m₀ = 0.5, n₀ = 1.0
▶ Running: α = 0.1, λ = 0.1, m₀ = 1.0, n₀ = 0.5


...

❌ Not enough valid front points.
❌ Failed: α = 100, λ = 0.8, m₀ = 0.05, n₀ = 1.0
▶ Running: α = 100, λ = 0.8, m₀ = 0.8, n₀ = 1.0
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 0.8, m₀ = 0.1, n₀ = 0.5
▶ Running: α = 100, λ = 0.8, m₀ = 0.9, n₀ = 0.5
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 0.8, m₀ = 0.1, n₀ = 1.0
▶ Running: α = 100, λ = 0.8, m₀ = 0.9, n₀ = 1.0
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 0.8, m₀ = 0.2, n₀ = 0.5
▶ Running: α = 100, λ = 0.8, m₀ = 0.95, n₀ = 0.5
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 0.8, m₀ = 0.2, n₀ = 1.0
▶ Running: α = 100, λ = 0.8, m₀ = 0.95, n₀ = 1.0
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 0.8, m₀ = 0.5, n₀ = 0.5
▶ Running: α = 100, λ = 0.8, m₀ = 1.0, n₀ = 0.5
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 0.8, m₀ = 0.5, n₀ = 1.0
▶ Running: α = 100, λ = 0.8, m₀ = 1.0, n₀ = 1.0
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 0.8, m₀ = 0.8, n₀ = 0.5
▶ Running: α = 100, λ

[Parallel(n_jobs=-1)]: Done 1552 tasks      | elapsed: 169.3min


❌ Not enough valid front points.
❌ Failed: α = 100, λ = 5, m₀ = 0.1, n₀ = 1.0
▶ Running: α = 100, λ = 5, m₀ = 0.9, n₀ = 0.5
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 5, m₀ = 0.1, n₀ = 0.5
▶ Running: α = 100, λ = 5, m₀ = 0.9, n₀ = 1.0
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 5, m₀ = 0.2, n₀ = 0.5
▶ Running: α = 100, λ = 5, m₀ = 0.95, n₀ = 0.5
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 5, m₀ = 0.2, n₀ = 1.0
▶ Running: α = 100, λ = 5, m₀ = 0.95, n₀ = 1.0
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 5, m₀ = 0.5, n₀ = 0.5
▶ Running: α = 100, λ = 5, m₀ = 1.0, n₀ = 0.5
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 5, m₀ = 0.5, n₀ = 1.0
▶ Running: α = 100, λ = 5, m₀ = 1.0, n₀ = 1.0
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 5, m₀ = 0.8, n₀ = 0.5
▶ Running: α = 100, λ = 10, m₀ = 0, n₀ = 0.5
❌ Not enough valid front points.
❌ Failed: α = 100, λ = 5, m₀ = 0.8, n₀ = 1.0
▶ Running: α = 100, λ = 10, m₀ = 0, n₀ = 1.0
❌ Not en

[Parallel(n_jobs=-1)]: Done 1620 out of 1620 | elapsed: 178.8min finished



❌ The following runs failed to compute wave speed:
  α = 0.1, λ = 0.1, m₀ = 0.8, n₀ = 0.5
  α = 0.1, λ = 0.1, m₀ = 0.8, n₀ = 1.0
  α = 0.1, λ = 0.1, m₀ = 0.9, n₀ = 0.5
  α = 0.1, λ = 0.1, m₀ = 0.9, n₀ = 1.0
  α = 0.1, λ = 0.1, m₀ = 0.95, n₀ = 0.5
  α = 0.1, λ = 0.1, m₀ = 0.95, n₀ = 1.0
  α = 0.1, λ = 0.1, m₀ = 1.0, n₀ = 0.5
  α = 0.1, λ = 0.1, m₀ = 1.0, n₀ = 1.0
  α = 0.1, λ = 0.2, m₀ = 0.8, n₀ = 0.5
  α = 0.1, λ = 0.2, m₀ = 0.8, n₀ = 1.0
  α = 0.1, λ = 0.2, m₀ = 0.9, n₀ = 0.5
  α = 0.1, λ = 0.2, m₀ = 0.9, n₀ = 1.0
  α = 0.1, λ = 0.2, m₀ = 0.95, n₀ = 0.5
  α = 0.1, λ = 0.2, m₀ = 0.95, n₀ = 1.0
  α = 0.1, λ = 0.2, m₀ = 1.0, n₀ = 0.5
  α = 0.1, λ = 0.2, m₀ = 1.0, n₀ = 1.0
  α = 0.1, λ = 0.5, m₀ = 0.8, n₀ = 0.5
  α = 0.1, λ = 0.5, m₀ = 0.8, n₀ = 1.0
  α = 0.1, λ = 0.5, m₀ = 0.9, n₀ = 0.5
  α = 0.1, λ = 0.5, m₀ = 0.9, n₀ = 1.0
  α = 0.1, λ = 0.5, m₀ = 0.95, n₀ = 0.5
  α = 0.1, λ = 0.5, m₀ = 0.95, n₀ = 1.0
  α = 0.1, λ = 0.5, m₀ = 1.0, n₀ = 0.5
  α = 0.1, λ = 0.5, m₀ = 1.0, n₀ = 1.0
  α = 